# OpenAPI APIを Bedrock AgentCore Gatewayを使ってMCPツールに変換する

## 概要
顧客は OpenAPI 仕様を JSON または YAML 形式で持ち込み、Bedrock AgentCore Gateway を使って API を MCP ツールに変換することができます。NASA の Open API を呼び出す Mars Weather エージェントを構築する方法を示します。

Gateway のワークフローには、エージェントを外部ツールに接続するための以下のステップが含まれます:
* **Gateway 用のツールを作成する** - OpenAPI 仕様などのスキーマを使ってツールを定義します。OpenAPI 仕様は Amazon Bedrock AgentCore によって解析され、Gateway が作成されます。
* **Gateway エンドポイントを作成する** - 受信認証付きの MCP 入力ポイントとなる Gateway を作成します。
* **Gateway にターゲットを追加する** - Gateway がリクエストを特定のツールにルーティングする方法を定義する OpenAPI ターゲットを設定します。OpenAPI ファイルの一部のすべての API は MCP 互換のツールになり、Gateway エンドポイント URL 経由で利用可能になります。各 OpenAPI Gateway ターゲットの送信認証を設定します。
* **エージェントコードを更新する** - エージェントを Gateway エンドポイントに接続し、統一された MCP インターフェースを介してすべての設定済みツールにアクセスできるようにします。

![How does it work](images/openapi-gateway-apikey.png)

### チュートリアルの詳細

| 情報                 | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルの種類 | インタラクティブ                                          |
| AgentCore コンポーネント | AgentCore Gateway、AgentCore Identity                  |
| Agentic フレームワーク | Strands Agents                                           |
| Gateway ターゲットの種類 | OpenAPI                                                |
| エージェント         | Mars Weather エージェント                                 |
| 受信認証 IdP         | Amazon Cognito                                            |
| 送信認証             | API キー                                                  |
| LLM モデル           | Anthropic Claude Sonnet 3.7、Amazon Nova Pro             |
| チュートリアルコンポーネント | AgentCore Gateway の作成と呼び出し                  |
| チュートリアルの分野 | 分野横断                                                  |
| 例の複雑さ           | 簡単                                                      |
| 使用した SDK         | boto3                                                     |

チュートリアルの最初の部分では、いくつかの AmazonCore Gateway ターゲットを作成します。

### チュートリアルのアーキテクチャ
このチュートリアルでは、OpenAPI yaml/json ファイルで定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。
デモの目的で、火星の天気に関する質問に答える Mars Weather エージェントを構築します。このエージェントは NASA の Open API を使用します。ソリューションでは、Amazon Bedrock モデルを使用する Strands エージェントを使用しています。
この例では、getInsightWeather ツールを持つ非常に単純なエージェントを使用します。

## 前提条件

このチュートリアルを実行するには、次のものが必要です。
* Jupyter notebook (Python kernel)
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
# Make sure you download the latest botocore and boto3 libraries.

日本語訳:

最新の botocore と boto3 ライブラリをダウンロードしていることを確認してください。
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook

# Amazon SageMaker ノートブックを使用していない場合は、AWS 認証情報を設定します

import boto3

# 認証情報を設定
session = boto3.Session(profile_name='default')
credentials = session.get_credentials()

# 認証情報を環境変数に設定
env = os.environ.copy()
env['AWS_ACCESS_KEY_ID'] = credentials.access_key
env['AWS_SECRET_ACCESS_KEY'] = credentials.secret_key
env['AWS_SESSION_TOKEN'] = credentials.token
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
import os
import sys

# Get the directory of the current script

現在のスクリプトのディレクトリを取得します。

日本語訳:
# 現在のスクリプトのディレクトリを取得する

import os

script_dir = os.path.dirname(__file__)

print(f"This script is located in: {script_dir}")

# Output:
# This script is located in: /path/to/current/directory
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

__file__ が定義されていない場合の代替手段 (例: Jupyter)

# Navigate to the directory containing utils.py (one level up)

utils.py を含むディレクトリに移動します (1 レベル上)
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]

日本語訳:

# sys.pathに追加する

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入する
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語はそのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]
sys.path.insert(0, utils_dir)

# Now you can import utils

日本語訳:
# 今から utils をインポートできます
import utils

In [ ]:
#### Gateway が引き受ける IAM ロールを作成する
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Amazon Cognito プールを作成して Gateway への受信認証を行う

resource "aws_cognito_user_pool" "pool" {
  name = "gateway-user-pool"
}

resource "aws_cognito_user_pool_client" "client" {
  name                                 = "gateway-user-pool-client"
  user_pool_id                         = aws_cognito_user_pool.pool.id
  generate_secret                      = false
  allowed_oauth_flows_user_pool_client = true
  allowed_oauth_flows                  = ["code", "implicit"]
  callback_urls                        = ["https://example.com/oauth2/idpresponse"]
  logout_urls                          = ["https://example.com"]
}

resource "aws_apigatewayv2_authorizer" "cognito" {
  api_id           = aws_apigatewayv2_api.gateway.id
  authorizer_type  = "JWT"
  identity_sources = ["$request.header.Authorization"]
  name             = "cognito-authorizer"

  jwt_configuration {
    audience = [aws_cognito_user_pool_client.client.id]
    issuer   = "https://${aws_cognito_user_pool.pool.endpoint}"
  }
}

In [ ]:
# Creating Cognito User Pool

日本語訳:

# Cognito ユーザープールの作成

Amazon Cognito は、Web アプリやモバイルアプリに認証機能を簡単に追加できるサービスです。ユーザープールを作成することで、ユーザー登録、ログイン、アクセストークンの発行などの機能を利用できます。

1. AWS マネジメントコンソールにログインし、Cognito サービスを選択します。
2. 「ユーザープールの管理」をクリックし、「新しいユーザープールを作成」を選びます。
3. ユーザープールの名前を入力し、「ステップを確認」をクリックします。
4. 「属性」セクションで、ユーザープロファイルに含める属性を選択します。一般的には email が必須です。
5. 「ポリシー」セクションでパスワードの複雑さなどを設定します。
6. 「MFA と確認済みの属性」セクションで、Multi-Factor Authentication (MFA) の設定を行います。
7. 「メッセージのカスタマイズ」セクションで、ユーザーに送信されるメールの内容をカスタマイズできます。
8. 最後に「ユーザープールの確認」をクリックし、設定を確認して「ユーザープールの作成」をクリックします。

これで Cognito ユーザープールが作成されました。次に、アプリケーションとの統合を行う必要があります。アプリケーションから AWS SDK や Amplify などを使って、ユーザープールにアクセスし、認証機能を実装します。
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL

discovery_url = "https://sheets.googleapis.com/$discovery/rest?version=v4"

日本語訳:

# 検索 URL を取得

discovery_url = "https://sheets.googleapis.com/$discovery/rest?version=v4"
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# ゲートウェイを作成する

日本語訳:
# ゲートウェイを作成します

resource "aws_nat_gateway" "gw" {
  allocation_id = aws_eip.nat.id
  subnet_id     = aws_subnet.public_1.id

  tags = {
    Name = "NAT Gateway Public Subnet 1"
  }

  # To ensure proper ordering, it is recommended to add an explicit dependency
  # on the Internet Gateway for the VPC.
  depends_on = [aws_internet_gateway.gw]
}

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step

日本語訳:

# CMK を使用せずに Cognito 認証者で CreateGateway を作成します。前のステップで作成した Cognito ユーザープールを使用してください。
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g

日本語訳:
# Client は Cognito で設定された ClientId と一致する必要があります。例: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoGWOpenAPIAPIKeyNasaOAI',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールには Gateway を作成/一覧表示/取得/削除する権限が必要です。

日本語訳:
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with OpenAPI target'
)
print(create_response)
# GatewayTarget作成に使用されるGatewayIDを取得する

data "aws_ec2_transit_gateway" "example" {
  transit_gateway_id = var.transit_gateway_id
}

resource "aws_ec2_transit_gateway_route_table" "example" {
  transit_gateway_id = data.aws_ec2_transit_gateway.example.id
}

resource "aws_ram_resource_share" "example" {
  name                      = "example-tgw-share"
  allow_external_principals = true
}

resource "aws_ram_principal_association" "example" {
  principal          = data.aws_caller_identity.example.account_id
  resource_share_arn = aws_ram_resource_share.example.arn
}

resource "aws_ram_resource_association" "example" {
  resource_arn       = aws_ec2_transit_gateway.example.arn
  resource_share_arn = aws_ram_resource_share.example.arn
}

data "aws_ram_resource_share_accepters" "example" {
  resource_share_arn = aws_ram_resource_share.example.arn

  filter {
    name   = "resource-share-arn"
    values = [aws_ram_resource_share.example.arn]
  }
}

output "gateway_id" {
  value       = data.aws_ec2_transit_gateway.example.id
  description = "GatewayID used for GatewayTarget creation"
}
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# NASA Open APIをBedrock AgentCore Gatewayを使ってMCPツールに変換する

日本語訳:

NASA Open API を Bedrock AgentCore Gateway を使って MCP ツールに変換する方法について説明します。

半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しました。

あなたは翻訳の専門家です。以下のテキストを英語から日本語に翻訳してください。
半角英数字の前後には半角スペースを挿入する。コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残してください。

翻訳するテキスト:
We are going to have a Mars Weather agent getting weather data from Nasa's Open APIs. You will need to register for Nasa Insight API [here](https://api.nasa.gov/). It's free! Once you register, you will get an API Key in your email. Use the API key to configure the credentials provider for creating the OpenAPI target.

日本語訳:
私たちは、Nasa の Open API から気象データを取得する Mars Weather agent を作ります。Nasa Insight API に [こちら](https://api.nasa.gov/) から登録する必要があります。無料です! 登録すると、メールで API Key が送られてきます。OpenAPI ターゲットを作成するために、credentials provider を設定する際にこの API key を使用してください。

In [ ]:
import boto3
from pprint import pprint
from botocore.config import Config

acps = boto3.client(service_name="bedrock-agentcore-control")

response=acps.create_api_key_credential_provider(
    name="NasaInsightAPIKey", 
    apiKey="", # Get an API key by signing up at api.nasa.gov. Takes 2-min to get an API key in your email.

日本語訳:

api.nasa.gov に登録すると、 API キー を取得できます。メールで API キー を受け取るまでに 2 分 ほどかかります。
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress Credentials provider ARN, {credentialProviderARN}")

# OpenAPI ターゲットを作成する

日本語訳:
OpenAPI は、 API の設計と文書化のための業界標準の仕様です。 OpenAPI ターゲットを作成すると、 API Gateway が OpenAPI 定義に基づいて API をデプロイします。

OpenAPI ターゲットを作成するには、以下の手順に従います。

1. AWS Management Console にサインインし、 API Gateway コンソールを開きます。
2. 「 REST API の作成」を選択します。
3. 「新しい API」を選択します。
4. 「OpenAPI」を選択します。
5. OpenAPI 定義本文に OpenAPI 定義を入力するか、またはインポートします。
6. 「インポート」をクリックします。

OpenAPI 定義がインポートされると、 API Gateway が OpenAPI 定義に基づいて API を作成します。次に、この API をデプロイしてリソースを作成し、クライアントがアクセスできるようにする必要があります。

#### S3 に NASA Open API の json ファイルをアップロードする

日本語訳:

NASA Open API の json ファイルを Amazon S3 にアップロードする手順は次のとおりです。

1. AWS マネジメントコンソールにログインし、S3 コンソールを開きます。
2. バケットを作成するか、既存のバケットを選択します。
3. バケットにファイルをアップロードするには、「アップロード」ボタンをクリックします。
4. ファイルを選択するウィンドウが開くので、アップロードする NASA Open API の json ファイルを選択します。
5. 「次へ」をクリックして、アップロードオプションを設定します (必要に応じて)。
6. 「アップロード」をクリックすると、ファイルがS3バケットにアップロードされます。

アップロード後、ファイルのリンクやパスを控えておきます。次のステップでこの情報が必要になります。

In [ ]:
# Create an S3 client

日本語訳:
S3 クライアントを作成します。

import boto3

s3 = boto3.client('s3')

# Upload a file to S3
s3.upload_file('/tmp/sample_file.txt', 'my-bucket', 'sample_file.txt')

# Download a file from S3
s3.download_file('my-bucket', 'sample_file.txt', '/tmp/sample_file.txt')
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWS アカウント ID と リージョンを取得する

account_id = subprocess.run(["aws", "sts", "get-caller-identity", "--query", "Account", "--output", "text"], stdout=subprocess.PIPE).stdout.decode('utf-8').strip()
region = subprocess.run(["aws", "configure", "get", "region"], stdout=subprocess.PIPE).stdout.decode('utf-8').strip()

print(f"AWS アカウント ID: {account_id}")
print(f"リージョン: {region}")
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters

パラメータを定義します。

num_epochs = 10 
batch_size = 32
learning_rate = 0.001

# Import necessary libraries

必要なライブラリをインポートします。

import numpy as np
import tensorflow as tf

# Load data

データを読み込みます。

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess data

データを前処理します。

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Define model

モデルを定義します。

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile model

モデルをコンパイルします。

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model

モデルを訓練します。

model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=1)

# Evaluate model

モデルを評価します。

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {test_acc * 100:.2f}%')
bucket_name = '' # Your s3 bucket to upload the OpenAPI json file.

日本語訳:

# OpenAPI json ファイルをアップロードする s3 バケットを指定してください。
file_path = 'openapi-specs/nasa_mars_insights_openapi.json'
object_key = 'nasa_mars_insights_openapi.json'
# ファイルをput_objectを使ってアップロードし、レスポンスを読む

日本語訳:

ファイルを put_object を使ってアップロードし、レスポンスを読みます。
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # アカウントIDとリージョンでアップロードされたオブジェクトの ARN を構築する

日本語訳:

アカウント ID と リージョンを使用して、アップロードされたオブジェクトの Amazon リソースネーム (ARN) を構築します。技術的な用語 (ARN、アカウント ID、リージョン) はそのまま残しました。
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

以下が日本語訳になります。

#### 送信認証を構成し、ゲートウェイターゲットを作成する

To configure outbound auth, run the following command:

```
$ kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.16/samples/sleep/sleep.yaml
```

This will deploy the sleep ControlPlatform which you can use as a test source for outbound traffic.

Next, create a gateway to handle the egress traffic:

```yaml
apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: istio-egressgateway
spec:
  selector:
    istio: egressgateway
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"
```

Create a `VirtualService` to route from the sidecars to the gateway:

```yaml
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: direct-through-egress-gateway
spec:
  hosts:
  - "*"
  gateways:
  - istio-egressgateway
  http:
  - match:
    - gateways:
      - istio-egressgateway
      port: 80
    route:
    - destination:
        host: istio-egressgateway.istio-system.svc.cluster.local
        port:
          number: 80
      weight: 100
```

In [ ]:
# S3 Uri for OpenAPI spec file

S3 URI は OpenAPI 仕様ファイルの場所を指定します。
nasa_openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

# API キーの認証情報プロバイダーの設定

日本語訳:

API キーの認証情報プロバイダーを設定するには、次の手順に従ってください。

1. AWS マネジメントコンソールにサインインし、IAM コンソールを開きます。

2. 左側のナビゲーションペインで [ Access keys ] (アクセスキー) を選択します。

3. [ Create New Access Key ] (新しいアクセスキーの作成) ボタンをクリックします。

4. 新しいアクセスキーのダイアログボックスが表示されたら、[ Download Key File ] (キーファイルのダウンロード) をクリックして、キーペアを安全な場所に保存します。このキーペアを使用して、AWS リソースにプログラムでアクセスできます。

5. キーペアをダウンロードしたら、[ Close ] (閉じる) をクリックします。アクセスキー ID とシークレットアクセスキーは、後で AWS CLI や SDK で使用できます。

6. 次に、credentials ファイルを作成または編集する必要があります。Linux や macOS の場合は ~/.aws/credentials、Windows の場合は C:\Users\YOUR_USER_NAME\.aws\credentials にあります。

7. credentials ファイルを開き、次の形式で新しいプロファイルを追加します。

[プロファイル名]
aws_access_key_id = YOUR_ACCESS_KEY_ID  
aws_secret_access_key = YOUR_SECRET_ACCESS_KEY

8. YOUR_ACCESS_KEY_ID と YOUR_SECRET_ACCESS_KEY は、前の手順でダウンロードしたキーペアの値に置き換えてください。

9. ファイルを保存して閉じます。

これで API キーの認証情報プロバイダーが設定されました。AWS CLI や SDK を使用する際は、--profile オプションでこのプロファイル名を指定することで、設定した認証情報を使用できます。
api_key_credential_config = [
    {
        "credentialProviderType" : "API_KEY", 
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                    "credentialParameterName": "api_key", # この行は、それぞれの API プロバイダーが期待する API キー名に置き換えてください。ヘッダーにトークンを渡す場合は、"Authorization" を使用してください。
                    "providerArn": credentialProviderARN,
                    "credentialLocation":"QUERY_PARAMETER", # api キーの場所。可能な値は "HEADER" と "QUERY_PARAMETER" です。

日本語訳:
# api キーの位置。可能な値は "HEADER" と "QUERY_PARAMETER" です。
                    #"credentialPrefix": " " # トークンの接頭辞。有効な値は "Basic" です。トークンにのみ適用されます。

日本語訳:
#"credentialPrefix": " " # トークンの 接頭辞 です。有効な値は "Basic" です。トークンにのみ適用されます。
            }
        }
    }
  ]

targetname='DemoOpenAPITargetS3NasaMars'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=nasa_openapi_s3_target_config,
    credentialProviderConfigurations=api_key_credential_config)

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装する Bedrock AgentCore Gateway を通じて、AWS ツールとシームレスに統合されています。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

Bedrock AgentCore Gateway の中核は、基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠の Gateway です。これらの API を使用すると、MCP 準拠のクライアントまたは SDK から、利用可能なツールを安全で標準化された方法で検出およびやり取りできます。Strands エージェントが AWS サービスにアクセスする必要がある場合は、これらの MCP 標準化されたエンドポイントを使用して Gateway と通信します。

Gateway の実装は、(MCP Authorization 仕様)[https://modelcontextprotocol.org/specification/draft/basic/authorization] を厳密に順守しており、堅牢なセキュリティとアクセス制御が保証されています。つまり、Strands エージェントによるすべてのツール呼び出しは、セキュリティを維持しながら強力な機能を可能にする認証ステップを経ます。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを検出し、次に InvokeTools を使用して特定のアクションを実行します。Gateway は、必要なすべてのセキュリティ検証、プロトコル変換、およびサービス対話を処理するため、プロセス全体がシームレスかつ安全に行われます。

このアーキテクチャアプローチにより、MCP 仕様を実装するクライアントまたは SDK は、Gateway を介して AWS サービスとやり取りできるため、AI エージェントの統合に対して汎用性と将来性のある解決策となります。

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# Amazon Cognitoからアクセストークンを要求する (inbound認証用)

日本語訳:

Amazon Cognito から inbound 認証用のアクセストークンを要求します。この処理は、以下の手順で行います。

1. Amazon Cognito の認証情報 (client_id、client_secret) を使って、 AWS 認証情報プロバイダーにアクセストークンを要求します。

2. 認証情報プロバイダーから受け取った access_token と id_token を確認します。

3. Amazon Cognito の user_info エンドポイントに access_token を送信し、ユーザー情報を取得します。

4. ユーザー情報から必要な値 (例: username、email など) を抽出し、後続の処理に使用します。

この手順を経ることで、Amazon Cognito 認証済みユーザーのアクセストークンと関連情報を安全に取得できます。

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# NASA Open APIを使ってBedrock AgentCore Gatewayを呼び出し、火星の天気エージェントに問い合わせる

日本語訳:

NASA Open API を使って Bedrock AgentCore Gateway を呼び出すことで、火星の天気エージェントに問い合わせることができます。 Bedrock AgentCore Gateway は、 API を呼び出して外部サービスとやり取りするためのゲートウェイです。この例では、NASA の公開 API を使って火星の気象データを取得します。

以下の手順で実行できます:

1. まず、 AgentScript を作成します。これは Bedrock AgentCore Gateway に対して実行したい一連の手順を記述したスクリプトです。

```python
# marsweather.agentscript

from bedrock_agentcore_gateway import AgentCoreGateway

gateway = AgentCoreGateway()

def get_mars_weather():
    response = gateway.request("https://api.nasa.gov/insight_weather/?api_key=DEMO_KEY&feedtype=json&ver=1.0")
    return response.json()

result = get_mars_weather()
print(result)
```

2. 次に、 Bedrock AgentCore Gateway を起動し、作成した AgentScript を実行します。

```bash
$ bedrock-agentcore-gateway run marsweather.agentscript
```

3. 実行すると、NASA の API から火星の気象データが取得され、結果が出力されます。

このように、Bedrock AgentCore Gateway を使うことで、外部の API やサービスを簡単に呼び出すことができます。AgentScript の中で任意のコードを記述でき、複雑なタスクを実行することも可能です。

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

以下が日本語訳になります。

## ~/.aws/credentials に設定された IAM グループ/ユーザーには Bedrock モデルへのアクセス権限が必要です
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.

日本語訳:

# ルート strands ロガーを設定します。問題をデバッグしている場合は、これを DEBUG に変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを確認するためのハンドラーを追加する

logs = run("kubectl logs deployment/nginx -n ingress-nginx", hide=True)

if logs.failed:
    failed("kubectl logs failed")

logs_output = logs.stdout.decode()
print(f"Logs for nginx: \n{logs_output}")

# Check for any errors in the logs
nginx_errors = [
    "error", 
    "err",
    "failed"
]

if any(error_msg in logs_output.lower() for error_msg in nginx_errors):
    failed("There are errors in the nginx logs")
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 

listTools 関数を呼び出します。
    tools = client.list_tools_sync()
    # エージェントを model と tools で作成する

日本語訳:

# エージェントを model と tools で作成する

agent = Agent(model, tools)
    agent = Agent(model=yourmodel,tools=tools) ## お好みのモデルに置き換えることができます
    print(f"Tools loaded in the agent are {agent.tool_names}")
    print(f"Tools configuration in the agent are {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.

サンプルのプロンプトでエージェントを呼び出します。これは MCP listTools のみを呼び出し、LLM がアクセスできるツールのリストを取得します。以下では実際にはツールを呼び出しません。
    agent("Hi , can you list all tools available to you")
    agent("What is the weather in northern part of the mars")
    # Invoke the agent with sample prompt, invoke the tool and display the response

サンプルのプロンプトでエージェントを呼び出し、ツールを呼び出して、応答を表示します。
    #MCPツールを明示的に呼び出します。MCPツールの名前と引数は、AWS Lambdaファンクションまたは OpenAPI/Smithy API と一致する必要があります。

日本語訳:
MCPツールを明示的に呼び出します。MCPツールの名前と引数は、あなたの AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります。
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # You can replace this with unique identifier.

日本語訳:

# これは一意の識別子に置き換えることができます。
    name=targetname+"___getInsightWeather", # This is the tool name based on AWS Lambda target types. This will change based on the target name

日本語訳:
これは AWS Lambda のターゲットタイプに基づくツール名です。ターゲット名に応じて変更されます。
    arguments={"ver": "1.0","feedtype": "json"}
    )
    # ログを確認するためのハンドラーを追加する

logs = run("kubectl logs deployment/nginx -n ingress-nginx", hide=True)

if logs.failed:
    failed("kubectl logs failed")

logs_output = logs.stdout.decode()
print(f"Logs for nginx: \n{logs_output}")

# Check for any errors in the logs
nginx_errors = [
    "error", 
    "err",
    "failed"
]

if any(error_msg in logs_output.lower() for error_msg in nginx_errors):
    failed("There are errors in the nginx logs")0
    print(f"Tool Call result: {result['content'][0]['text']}")


**問題: 以下のセルを実行中に下記のエラーが発生した場合、pydantic と pydantic-core のバージョンが互換性がないことを示しています。**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```

**解決方法:**

pydantic==2.7.2 と pydantic-core 2.27.2 が互換性があることを確認する必要があります。確認後、カーネルを再起動してください。

# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、s3 バケットなどの追加リソースも作成されます。これらは、実行した例次第で、クリーンアップの一環として手動で削除する必要があるかもしれません。

## ゲートウェイを削除する (オプション)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)